In [18]:
%load_ext autoreload
%autoreload 2

import torch
from rl4co.utils.trainer import RL4COTrainer

import sys
sys.path.append(r'/data1/shy/zgc/llm_solver/routefinder')

from routefinder.models import RouteFinderBase, RouteFinderPolicy
from routefinder.envs.mtvrp import MTVRPEnv, MTVRPGenerator
from routefinder.utils import rollout, greedy_policy, evaluate

from LLM import LLM_api

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
llm = LLM_api(model="Qwen/Qwen2.5-7B-Instruct", key_idx=0)
text = llm.get_text(content="你好")

In [20]:
print(text)

你好！有什么问题我可以帮助你解答吗？


In [21]:

generator = MTVRPGenerator(num_loc=50, variant_preset="all")
env = MTVRPEnv(generator, check_solution=False)

# Generate data (mixed variants)
instance_num = 3

td_data = env.generator(instance_num)
variant_names = env.get_variant_names(td_data)

# print(td_data.keys())

for idx in range(len(td_data)):
    print(f"Variant: {variant_names[idx]}")
    # print(f"data: {td_data[idx]}")

Variant: VRPTW
Variant: VRPLTW
Variant: VRPBTW


/data1/shy/miniconda3/envs/py310/lib/python3.10/site-packages/torchrl/data/tensor_specs.py:6294: DeprecationWarning: The BoundedTensorSpec has been deprecated and will be removed in v0.8. Please use Bounded instead.
  warnings.warn(
/data1/shy/miniconda3/envs/py310/lib/python3.10/site-packages/torchrl/data/tensor_specs.py:6294: DeprecationWarning: The UnboundedDiscreteTensorSpec has been deprecated and will be removed in v0.8. Please use Unbounded instead.
  warnings.warn(
/data1/shy/miniconda3/envs/py310/lib/python3.10/site-packages/torchrl/data/tensor_specs.py:6294: DeprecationWarning: The CompositeSpec has been deprecated and will be removed in v0.8. Please use Composite instead.
  warnings.warn(
/data1/shy/miniconda3/envs/py310/lib/python3.10/site-packages/torchrl/data/tensor_specs.py:6294: DeprecationWarning: The UnboundedContinuousTensorSpec has been deprecated and will be removed in v0.8. Please use Unbounded instead.
  warnings.warn(


In [ ]:

generator = MTVRPGenerator(num_loc=50, variant_preset="all")
env = MTVRPEnv(generator, check_solution=False)

# Generate data (mixed variants)
instance_num = 3

td_data = env.generator(instance_num)
variant_names = env.get_variant_names(td_data)

print(td_data.keys())

for idx in range(len(td_data)):
    print(f"Variant: {variant_names[idx]}")
    print(f"data: {td_data[idx]}")

上面的代码随机构造了VRP变体的实例。variant_names为VRP变体问题的名称，td_data为实例具体数据，其key为['locs', 'demand_backhaul', 'demand_linehaul', 'backhaul_class', 'distance_limit', 'time_windows', 'service_time', 'vehicle_capacity', 'capacity_original', 'open_route', 'speed']。
下面我需要你编写python程序，调用LLM将每个VRP问题实例转化为自然语言表述。从prompt设计角度来看，首先你需要从设备维修、废物管理、应急响应、现场任务分配、移动监控、货物运送、广告投放、人员调度选择一个场景，然后让LLM根据该场景和问题实例数据，将结构化的VRP问题实例转为自然语言描述。LLM的调用可以使用：
llm = LLM_api(model="Qwen/Qwen2.5-7B-Instruct", key_idx=0)
text = llm.get_text(content="你好")

你需要额外注意几点:
1. LLM提示词中不要出现variant_name真值
2. 将每个实例的自然语言描述保存为样本，对应的variant_name为标签，保存一个数据集
3. 代码和prompt不要出现中文，请全部使用英文，包括注释

In [24]:
from LLM import LLM_api  # Assuming LLM API is available
import json
import numpy as np
import datetime

# Define a function to generate the natural language prompt
def generate_prompt(instance_data, scenario):
    prompt = f"You are a problem modeler and you are provided with an example of a VRP problem below, please translate it into a plain natural language description in {scenario} and output the description directly.\n"
    prompt += f"Please be careful not to give the specific type of problem directly in the description, and also your description should be complete enough to recover the problem instance.\n"
    prompt += f"Specifically, the problem has the following parameters:\n"
    prompt += f"Locations (depot + {len(instance_data['locs']) - 1} customers): {instance_data['locs']}\n"
    prompt += f"Backhaul Demand: {instance_data['demand_backhaul']}\n"
    prompt += f"Linehaul Demand: {instance_data['demand_linehaul']}\n"
    prompt += f"Backhaul Class: {instance_data['backhaul_class']}\n"
    prompt += f"Distance Limit: {instance_data['distance_limit']}\n"
    prompt += f"Time Windows: {instance_data['time_windows']}\n"
    prompt += f"Service Time: {instance_data['service_time']}\n"
    prompt += f"Vehicle Capacity: {instance_data['vehicle_capacity']}\n"
    prompt += f"Open Route: {instance_data['open_route']}\n"
    prompt += f"Speed: {instance_data['speed']}\n"
    return prompt

# Function to generate natural language descriptions using LLM
def generate_natural_language_description(instance_data, scenario, llm):
    # Generate the prompt
    prompt = generate_prompt(instance_data, scenario)
    
    # Request the LLM to generate the description
    response = llm.get_text(content=prompt)
    return response

# Function to save the generated descriptions as a dataset
def save_to_dataset(descriptions, variant_namesm, pclass = 'VRP'):
    time_stamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f"{pclass}_{time_stamp}.json"
    data = [{"variant_name": variant_names[i], "description": descriptions[i]} for i in range(len(descriptions))]
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

# Assuming you have the following setup
generator = MTVRPGenerator(num_loc=10, variant_preset="all")
env = MTVRPEnv(generator, check_solution=False)

# Define the real-world scenario (choose one from the options
scenario_list = ['field service management', 'waste collection', 'emergency response', 'mobile surveillance', 'goods delivery', 'advertising placement', 'personnel scheduling']

# Generate data (mixed variants)
instance_num = 10
td_data = env.generator(instance_num)
variant_names = env.get_variant_names(td_data)
scenarios = np.random.choice(scenario_list, instance_num)

# LLM API initialization
llm = LLM_api(model="Qwen/Qwen2.5-7B-Instruct", key_idx=0)

# List to store descriptions
descriptions = []

# Loop through each instance and generate a description
for idx in range(len(td_data)):
    
    print(f"Processing variant {variant_names[idx]}...")
    
    description = generate_natural_language_description(td_data[idx], scenarios[idx], llm)
    
    # Print or save the description
    print(f"Description: {description}")
    
    # Append the description to the list
    descriptions.append(description)

# Save the descriptions to a file
save_to_dataset(descriptions, variant_names)

/data1/shy/miniconda3/envs/py310/lib/python3.10/site-packages/torchrl/data/tensor_specs.py:6294: DeprecationWarning: The BoundedTensorSpec has been deprecated and will be removed in v0.8. Please use Bounded instead.
  warnings.warn(
/data1/shy/miniconda3/envs/py310/lib/python3.10/site-packages/torchrl/data/tensor_specs.py:6294: DeprecationWarning: The UnboundedDiscreteTensorSpec has been deprecated and will be removed in v0.8. Please use Unbounded instead.
  warnings.warn(
/data1/shy/miniconda3/envs/py310/lib/python3.10/site-packages/torchrl/data/tensor_specs.py:6294: DeprecationWarning: The CompositeSpec has been deprecated and will be removed in v0.8. Please use Composite instead.
  warnings.warn(
/data1/shy/miniconda3/envs/py310/lib/python3.10/site-packages/torchrl/data/tensor_specs.py:6294: DeprecationWarning: The UnboundedContinuousTensorSpec has been deprecated and will be removed in v0.8. Please use Unbounded instead.
  warnings.warn(


Processing variant OVRP...
Description: In a field service management scenario, there is a depot and 10 customer locations that need to be serviced. The locations of these sites are as follows:

1. (0.7896, 0.1726)
2. (0.7137, 0.5533)
3. (0.1077, 0.5573)
4. (0.5846, 0.1763)
5. (0.4229, 0.3828)
6. (0.2761, 0.7425)
7. (0.9574, 0.4337)
8. (0.5405, 0.0195)
9. (0.4389, 0.6287)
10. (0.5259, 0.8844)
11. (0.5563, 0.6948)

Each customer has a linehaul demand, which is the amount of service required, as follows:

1. 0.2333
2. 0.2000
3. 0.1667
4. 0.2000
5. 0.0333
6. 0.1667
7. 0.1000
8. 0.0333
9. 0.1333
10. 0.2333

All customers have a backhaul demand of 0, meaning there is no demand for backhaul services. The service time at each customer is 0, and there are no specific time windows or speed constraints. Each customer is located within an infinite distance limit, and there are no specific service time windows or backhaul classes. The vehicles have a capacity of 1 and can start from the depot. The

In [ ]:
import json
from Agent import Classifier, Checker
from LLM import LLM_api
import time

# Initialize the LLM API
llm = LLM_api(model="Qwen/QwQ-32B", key_idx=0)

# Initialize the Classifier and Checker
classifier = Classifier(llm)
checker = Checker(llm)

# Load the dataset of VRP problem descriptions and their labels
def load_dataset(filename="VRP_20250312162103.json"):
    with open(filename, 'r') as file:
        return json.load(file)

# Function to calculate the success rate
def calculate_success_rate(dataset):
    correct_predictions = 0
    total_predictions = len(dataset)
    print(f"Total predictions: {total_predictions}")

    # Iterate through the dataset and classify each VRP problem
    for data in dataset:
        variant_name = data["variant_name"]
        problem_desc = data["description"]
        print(variant_name)

        max_rounds = 5  # Limit the maximum number of iterations to avoid infinite loops

        for _ in range(max_rounds):
            # Classify the VRP problem
            classification = classifier.run(problem_desc)

            # Check the classification
            is_correct, reason = checker.run(problem_desc, classification)

            if is_correct:
                print(f"\n[Final Output] VRP Problem Type: {classification}")
                if classification == variant_name:
                    correct_predictions += 1
                break
            else:
                print(f"\n[Checker] The classification is incorrect. Reason: {reason}")
                # Append the reason to the problem description, so the classifier can reconsider
                problem_desc += f"\n(Checker feedback: {reason})"
                time.sleep(1)  # Delay to avoid rapid repetitive calls
        else:
            # If we exit the loop without agreement
            print("[Warning] Reached the maximum number of iterations without agreement.")

    # Calculate the success rate
    success_rate = correct_predictions / total_predictions
    return success_rate

# Load the dataset
dataset = load_dataset("VRP_20250312162103.json")

# Calculate the success rate
success_rate = calculate_success_rate(dataset)

# Print the success rate
print(f"\nLLM Classification Success Rate: {success_rate * 100:.2f}%")

Total predictions: 10
OVRP

[Final Output] VRP Problem Type: OVRP
VRPBL

[Final Output] VRP Problem Type: VRPBL
OVRP

[Final Output] VRP Problem Type: CVRP
OVRPBLTW
